In [1]:
import argparse
import collections
import sklearn.metrics as metrics
import tensorboardX as tb
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.modules.loss as loss
import torch.optim as optim
import torch.utils as utils
import data
import my

In [2]:
args = argparse.Namespace()
args.batch_size = 50
args.gpu = 1
args.log_every = 1
args.n_iterations = 5000

keys = sorted(vars(args).keys())
excluded = ('gpu', 'log_every', 'n_iterations')
run_id = 'cifar10-9-1-ce-linear-' + '-'.join('%s-%s' % (key, str(getattr(args, key))) for key in keys if key not in excluded)
writer = tb.SummaryWriter('runs/' + run_id)

In [3]:
if args.gpu < 0:
    cuda = False
else:
    cuda = True
    th.cuda.set_device(args.gpu)

labelling = {(0, 9) : 0, (9, 10) : 1}
# labelling = {(0, 5) : 0, (5, 6) : 1, (6, 7) : 2, (7, 8) : 3, (8, 9) : 4, (9, 10) : 5}
train_x, train_y, test_x, test_y = data.load_cifar10(labelling, torch=True)

train_set = utils.data.TensorDataset(train_x, train_y)
train_loader = utils.data.DataLoader(train_set, 4096, drop_last=False)
test_set = utils.data.TensorDataset(test_x, test_y)
test_loader = utils.data.DataLoader(test_set, 4096, drop_last=False)

loader = data.BalancedDataLoader(train_x, train_y, args.batch_size, cuda)

n_classes = int(train_y.max() - train_y.min() + 1)

In [4]:
def global_scores(c, loader, average='binary'):
    keys = ('accuracy', 'precision', 'recall', 'f1')
    scores = (
        metrics.accuracy_score,
        lambda y, y_bar: metrics.precision_score(y, y_bar, average=average),
        lambda y, y_bar: metrics.recall_score(y, y_bar, average=average),
        lambda y, y_bar: metrics.f1_score(y, y_bar, average=average),
    )
    values = [value.item() for value in my.global_scores(c, loader, scores)]
    return collections.OrderedDict(zip(keys, values))

In [5]:
c = nn.Linear(3 * 32 * 32, n_classes)

if cuda:
    c.cuda()
    
# optimizer = optim.SGD(c.parameters(), lr=1e-1, momentum=0.9)
optimizer = optim.Adam(c.parameters(), amsgrad=True)

for key, value in global_scores(c, test_loader).items():
    print(key, value)

accuracy 0.4851
precision 0.39
recall 0.07912355447352404
f1 0.13155675493337832


In [6]:
n_iterations = 0
for i in range(args.n_iterations):
    x, y = next(loader)
    ce = loss.CrossEntropyLoss()(c(x), y)
    optimizer.zero_grad()
    ce.backward()
    optimizer.step()

    if (i + 1) % args.log_every == 0:
        train_scores = global_scores(c, train_loader)
        test_scores = global_scores(c, test_loader)

        prefix = '0' * (len(str(args.n_iterations)) - len(str(i + 1)))
        print('[iteration %s%d]' % (prefix, i + 1) + \
              ' | '.join('%s %0.3f/%0.3f' % (key, value, test_scores[key]) for key, value in train_scores.items()))

        for key, value in train_scores.items():
            writer.add_scalar('train-' + key, value, i)

        for key, value in test_scores.items():
            writer.add_scalar('test-' + key, value, i)

[iteration 0001]accuracy 0.541/0.528 | precision 0.613/0.624 | recall 0.127/0.126 | f1 0.211/0.209
[iteration 0002]accuracy 0.541/0.524 | precision 0.622/0.614 | recall 0.129/0.123 | f1 0.213/0.205
[iteration 0003]accuracy 0.544/0.530 | precision 0.668/0.650 | recall 0.136/0.130 | f1 0.227/0.217
[iteration 0004]accuracy 0.499/0.497 | precision 0.631/0.596 | recall 0.120/0.114 | f1 0.201/0.192
[iteration 0005]accuracy 0.505/0.503 | precision 0.633/0.599 | recall 0.121/0.116 | f1 0.204/0.194
[iteration 0006]accuracy 0.515/0.516 | precision 0.638/0.599 | recall 0.124/0.119 | f1 0.208/0.198
[iteration 0007]accuracy 0.527/0.530 | precision 0.641/0.601 | recall 0.128/0.123 | f1 0.213/0.204
[iteration 0008]accuracy 0.548/0.543 | precision 0.667/0.639 | recall 0.137/0.132 | f1 0.228/0.219
[iteration 0009]accuracy 0.547/0.550 | precision 0.620/0.629 | recall 0.130/0.132 | f1 0.215/0.219
[iteration 0010]accuracy 0.547/0.550 | precision 0.591/0.605 | recall 0.125/0.129 | f1 0.207/0.212
[iteration

[iteration 0084]accuracy 0.565/0.570 | precision 0.532/0.517 | recall 0.121/0.119 | f1 0.197/0.194
[iteration 0085]accuracy 0.566/0.570 | precision 0.480/0.472 | recall 0.112/0.111 | f1 0.181/0.180
[iteration 0086]accuracy 0.573/0.580 | precision 0.461/0.477 | recall 0.110/0.115 | f1 0.178/0.185
[iteration 0087]accuracy 0.587/0.594 | precision 0.483/0.515 | recall 0.118/0.126 | f1 0.189/0.202
[iteration 0088]accuracy 0.601/0.609 | precision 0.519/0.546 | recall 0.129/0.136 | f1 0.206/0.218
[iteration 0089]accuracy 0.618/0.627 | precision 0.554/0.580 | recall 0.141/0.149 | f1 0.225/0.237
[iteration 0090]accuracy 0.626/0.635 | precision 0.593/0.614 | recall 0.151/0.158 | f1 0.241/0.252
[iteration 0091]accuracy 0.605/0.614 | precision 0.640/0.639 | recall 0.151/0.155 | f1 0.245/0.249
[iteration 0092]accuracy 0.615/0.617 | precision 0.711/0.703 | recall 0.167/0.166 | f1 0.270/0.269
[iteration 0093]accuracy 0.650/0.644 | precision 0.764/0.758 | recall 0.190/0.186 | f1 0.304/0.299
[iteration

[iteration 0167]accuracy 0.674/0.670 | precision 0.727/0.711 | recall 0.196/0.191 | f1 0.308/0.301
[iteration 0168]accuracy 0.702/0.696 | precision 0.747/0.732 | recall 0.215/0.209 | f1 0.334/0.325
[iteration 0169]accuracy 0.688/0.684 | precision 0.741/0.724 | recall 0.206/0.200 | f1 0.322/0.314
[iteration 0170]accuracy 0.669/0.666 | precision 0.714/0.695 | recall 0.191/0.186 | f1 0.301/0.294
[iteration 0171]accuracy 0.665/0.660 | precision 0.667/0.654 | recall 0.181/0.177 | f1 0.284/0.278
[iteration 0172]accuracy 0.684/0.684 | precision 0.631/0.613 | recall 0.184/0.181 | f1 0.285/0.279
[iteration 0173]accuracy 0.675/0.679 | precision 0.601/0.570 | recall 0.174/0.170 | f1 0.270/0.262
[iteration 0174]accuracy 0.623/0.630 | precision 0.587/0.561 | recall 0.149/0.147 | f1 0.238/0.233
[iteration 0175]accuracy 0.605/0.614 | precision 0.600/0.576 | recall 0.145/0.144 | f1 0.233/0.230
[iteration 0176]accuracy 0.613/0.619 | precision 0.636/0.607 | recall 0.154/0.151 | f1 0.248/0.242
[iteration

[iteration 0250]accuracy 0.680/0.690 | precision 0.558/0.548 | recall 0.168/0.171 | f1 0.258/0.261
[iteration 0251]accuracy 0.685/0.695 | precision 0.540/0.540 | recall 0.167/0.173 | f1 0.256/0.262
[iteration 0252]accuracy 0.702/0.712 | precision 0.530/0.528 | recall 0.175/0.180 | f1 0.263/0.268
[iteration 0253]accuracy 0.719/0.726 | precision 0.531/0.523 | recall 0.185/0.188 | f1 0.274/0.276
[iteration 0254]accuracy 0.706/0.716 | precision 0.538/0.551 | recall 0.178/0.187 | f1 0.268/0.280
[iteration 0255]accuracy 0.698/0.707 | precision 0.565/0.578 | recall 0.179/0.187 | f1 0.272/0.283
[iteration 0256]accuracy 0.702/0.712 | precision 0.573/0.583 | recall 0.183/0.191 | f1 0.278/0.288
[iteration 0257]accuracy 0.703/0.712 | precision 0.600/0.598 | recall 0.189/0.194 | f1 0.288/0.293
[iteration 0258]accuracy 0.712/0.719 | precision 0.626/0.628 | recall 0.200/0.205 | f1 0.303/0.309
[iteration 0259]accuracy 0.711/0.721 | precision 0.644/0.657 | recall 0.203/0.212 | f1 0.308/0.320
[iteration

[iteration 0333]accuracy 0.766/0.758 | precision 0.675/0.652 | recall 0.251/0.239 | f1 0.366/0.350
[iteration 0334]accuracy 0.795/0.787 | precision 0.659/0.642 | recall 0.278/0.266 | f1 0.391/0.377
[iteration 0335]accuracy 0.802/0.799 | precision 0.625/0.611 | recall 0.281/0.274 | f1 0.387/0.379
[iteration 0336]accuracy 0.802/0.798 | precision 0.620/0.599 | recall 0.279/0.270 | f1 0.385/0.373
[iteration 0337]accuracy 0.800/0.799 | precision 0.611/0.580 | recall 0.275/0.267 | f1 0.379/0.366
[iteration 0338]accuracy 0.781/0.783 | precision 0.588/0.571 | recall 0.249/0.247 | f1 0.350/0.345
[iteration 0339]accuracy 0.762/0.766 | precision 0.580/0.563 | recall 0.229/0.228 | f1 0.328/0.325
[iteration 0340]accuracy 0.793/0.792 | precision 0.616/0.583 | recall 0.268/0.259 | f1 0.373/0.359
[iteration 0341]accuracy 0.806/0.804 | precision 0.627/0.601 | recall 0.286/0.278 | f1 0.393/0.380
[iteration 0342]accuracy 0.803/0.799 | precision 0.650/0.615 | recall 0.286/0.274 | f1 0.398/0.379
[iteration

...

[iteration 4649]accuracy 0.887/0.880 | precision 0.367/0.340 | recall 0.423/0.389 | f1 0.393/0.363
[iteration 4650]accuracy 0.890/0.887 | precision 0.344/0.321 | recall 0.438/0.417 | f1 0.385/0.363
[iteration 4651]accuracy 0.893/0.888 | precision 0.306/0.274 | recall 0.451/0.411 | f1 0.364/0.329
[iteration 4652]accuracy 0.890/0.885 | precision 0.297/0.268 | recall 0.429/0.392 | f1 0.351/0.318
[iteration 4653]accuracy 0.881/0.876 | precision 0.291/0.266 | recall 0.378/0.347 | f1 0.329/0.301
[iteration 4654]accuracy 0.876/0.870 | precision 0.299/0.266 | recall 0.359/0.320 | f1 0.326/0.291
[iteration 4655]accuracy 0.880/0.873 | precision 0.315/0.282 | recall 0.378/0.337 | f1 0.344/0.307
[iteration 4656]accuracy 0.893/0.887 | precision 0.322/0.286 | recall 0.451/0.407 | f1 0.376/0.336
[iteration 4657]accuracy 0.900/0.894 | precision 0.333/0.296 | recall 0.501/0.457 | f1 0.400/0.359
[iteration 4658]accuracy 0.896/0.891 | precision 0.375/0.342 | recall 0.474/0.440 | f1 0.419/0.385
[iteration

[iteration 4732]accuracy 0.903/0.898 | precision 0.205/0.179 | recall 0.546/0.479 | f1 0.298/0.261
[iteration 4733]accuracy 0.904/0.901 | precision 0.173/0.156 | recall 0.565/0.513 | f1 0.265/0.239
[iteration 4734]accuracy 0.905/0.903 | precision 0.160/0.148 | recall 0.588/0.550 | f1 0.252/0.233
[iteration 4735]accuracy 0.905/0.903 | precision 0.160/0.148 | recall 0.594/0.554 | f1 0.252/0.234
[iteration 4736]accuracy 0.905/0.903 | precision 0.145/0.142 | recall 0.598/0.555 | f1 0.234/0.226
[iteration 4737]accuracy 0.904/0.902 | precision 0.132/0.128 | recall 0.584/0.538 | f1 0.216/0.207
[iteration 4738]accuracy 0.903/0.902 | precision 0.123/0.125 | recall 0.562/0.534 | f1 0.202/0.203
[iteration 4739]accuracy 0.902/0.901 | precision 0.125/0.123 | recall 0.548/0.517 | f1 0.204/0.199
[iteration 4740]accuracy 0.901/0.900 | precision 0.129/0.123 | recall 0.527/0.498 | f1 0.208/0.197
[iteration 4741]accuracy 0.902/0.901 | precision 0.141/0.137 | recall 0.546/0.519 | f1 0.224/0.217
[iteration

[iteration 4815]accuracy 0.905/0.902 | precision 0.237/0.205 | recall 0.565/0.520 | f1 0.334/0.294
[iteration 4816]accuracy 0.905/0.903 | precision 0.250/0.222 | recall 0.559/0.534 | f1 0.345/0.314
[iteration 4817]accuracy 0.905/0.903 | precision 0.274/0.249 | recall 0.550/0.538 | f1 0.366/0.340
[iteration 4818]accuracy 0.905/0.905 | precision 0.292/0.269 | recall 0.551/0.547 | f1 0.382/0.361
[iteration 4819]accuracy 0.906/0.904 | precision 0.309/0.287 | recall 0.551/0.537 | f1 0.396/0.374
[iteration 4820]accuracy 0.906/0.904 | precision 0.333/0.305 | recall 0.547/0.535 | f1 0.414/0.389
[iteration 4821]accuracy 0.905/0.904 | precision 0.355/0.326 | recall 0.541/0.536 | f1 0.429/0.405
[iteration 4822]accuracy 0.906/0.902 | precision 0.356/0.321 | recall 0.547/0.519 | f1 0.431/0.397
[iteration 4823]accuracy 0.905/0.902 | precision 0.364/0.327 | recall 0.538/0.514 | f1 0.435/0.400
[iteration 4824]accuracy 0.905/0.901 | precision 0.368/0.334 | recall 0.534/0.509 | f1 0.436/0.403
[iteration

[iteration 4898]accuracy 0.900/0.894 | precision 0.259/0.244 | recall 0.501/0.446 | f1 0.341/0.315
[iteration 4899]accuracy 0.903/0.895 | precision 0.289/0.260 | recall 0.526/0.460 | f1 0.373/0.332
[iteration 4900]accuracy 0.903/0.896 | precision 0.315/0.284 | recall 0.528/0.465 | f1 0.395/0.353
[iteration 4901]accuracy 0.900/0.894 | precision 0.320/0.282 | recall 0.504/0.449 | f1 0.391/0.346
[iteration 4902]accuracy 0.897/0.890 | precision 0.337/0.298 | recall 0.476/0.428 | f1 0.395/0.351
[iteration 4903]accuracy 0.895/0.887 | precision 0.335/0.298 | recall 0.465/0.413 | f1 0.389/0.346
[iteration 4904]accuracy 0.895/0.887 | precision 0.354/0.316 | recall 0.467/0.417 | f1 0.403/0.360
[iteration 4905]accuracy 0.896/0.888 | precision 0.364/0.323 | recall 0.472/0.423 | f1 0.411/0.366
[iteration 4906]accuracy 0.895/0.886 | precision 0.383/0.351 | recall 0.469/0.417 | f1 0.422/0.381
[iteration 4907]accuracy 0.891/0.882 | precision 0.390/0.356 | recall 0.447/0.398 | f1 0.417/0.376
[iteration

[iteration 4981]accuracy 0.895/0.892 | precision 0.242/0.244 | recall 0.458/0.430 | f1 0.317/0.311
[iteration 4982]accuracy 0.894/0.890 | precision 0.280/0.277 | recall 0.453/0.422 | f1 0.346/0.334
[iteration 4983]accuracy 0.894/0.889 | precision 0.321/0.312 | recall 0.456/0.424 | f1 0.376/0.359
[iteration 4984]accuracy 0.895/0.889 | precision 0.359/0.336 | recall 0.465/0.429 | f1 0.405/0.377
[iteration 4985]accuracy 0.897/0.890 | precision 0.374/0.342 | recall 0.483/0.438 | f1 0.422/0.384
[iteration 4986]accuracy 0.899/0.892 | precision 0.389/0.351 | recall 0.491/0.449 | f1 0.434/0.394
[iteration 4987]accuracy 0.899/0.893 | precision 0.392/0.352 | recall 0.493/0.455 | f1 0.437/0.397
[iteration 4988]accuracy 0.898/0.889 | precision 0.386/0.342 | recall 0.485/0.433 | f1 0.430/0.382
[iteration 4989]accuracy 0.892/0.886 | precision 0.354/0.322 | recall 0.450/0.410 | f1 0.396/0.361
[iteration 4990]accuracy 0.880/0.876 | precision 0.328/0.287 | recall 0.383/0.352 | f1 0.353/0.316
[iteration

In [ ]:
print(my.global_scores(c, train_loader, metrics.classification_report))
print(my.global_scores(c, test_loader, metrics.classification_report))